In [1]:
import sys
sys.path.append('../../')

In [2]:
preprocessed_dataset_path = 'preprocessed_dataset/'

In [3]:
from codes.docs.analysis.nimagen import stats, visualisation, graph
from codes.docs.analysis import data_preprocessing, genetic_file_preprocess

In [4]:
# from codes.docs.analysis import data_preprocessing, data_exploration, visualisation, model_training
# from codes.docs.analysis.gene_analysis import genetic_file_preprocess
# from codes.docs.analysis.MLtraining import ML_utils
# from codes.docs.analysis import graph_analysis

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
%matplotlib inline
import tqdm

import statsmodels.api as sm
import statsmodels.formula.api as sfm

from scipy.stats import ttest_ind, pearsonr,zscore
import pandas as pd
import re



Here, the volumetric dataset is preprocessed to group the same brain regions.\
Then mass univariate test is preformed, across multiple regions.

# Main analysis

In [6]:
european_volumetric_dataset_term = pd.read_csv('../../dataset/preprocessed_dataset/imaging_data/log_file/rel3/volumetric/european_volumetric_df_rel3_batch2.csv')

In [7]:
european_volumetric_dataset_term = european_volumetric_dataset_term[(european_volumetric_dataset_term['radiology_score']<5)&(european_volumetric_dataset_term['radiology_low_image_quality']==False)]

`european_volumetric_dataset_term` has 279 observations (after removal of low image quality and radiology score)

In [27]:
df = european_volumetric_dataset_term.copy()

ancestry = [f'euro_Anc_PC{i}'
            for i in range(1, 4)]  # remove any ancestry outliers

df = stats.MassUnivariate.remove_outliers(
    df,
    col=ancestry,
    threshold=3.5,
    remove_schemes='any',
    percentage_of_outlier=0.1)

In [28]:
batch2_ASD_PRS = genetic_file_preprocess.Cohort.preprocess_PRSice_PRS_Anc_files('../../codes/gene_set/PRSice/output/asd/ASD.all_score',column_prefix='ASD_PRS_')

In [29]:
df = df.drop(columns=[i for i in df.columns if 'ASD' in i])
df = df.merge(batch2_ASD_PRS,on='ID',how='left')

In [30]:
pca, prs_pca, loading_matrix = stats.FeatureReduction.perform_PCA(df,
                                                                dependentVar_cols=[i for i in df.columns if 'ASD_PRS' in i],scaling=True,n_components=1)

df['PRS_pca'] = prs_pca

In [31]:
df['ID/Session'] = 'sub-'+df['ID']+'/'+'ses-'+df['Session'].astype('str')

In [35]:
df[['ID/Session','sex','GA','PMA','TBV','PRS_pca']+ancestry].to_csv('../../codes/DrawEMVol/available_subjects_ASDPRSPCA.csv',index=False,header=False)